In [1]:
import pandas as pd
df = pd.read_pickle('newsis_local.pkl')

In [2]:
df.head(3)

,article,title,sec,sec_sub,len
14,제주지방기상청은 2일 오후 9시를 기해 제주도 서부앞바다에 풍랑주의보를 발효했다. ...,제주서부앞바다 오후 9시 풍랑주의보,전국,제주,233
21,강원 원주경찰서는 지난 1일 동거녀를 상습적으로 폭행한 혐의(폭력행위등처벌에 관한 ...,"원주경찰, 동거녀 상습폭행 40대 구속",전국,강원,308
23,2일 오후 12시18분께 충남 보령시 대천해수욕장 앞 인근 해상에서 이모(57·여·...,보령 대천해수욕장서 50대女 변사체 발견,전국,대전/충남,270


In [3]:
counts=df.groupby('sec_sub').size()
section_idx=enumerate(counts.keys())
section_to_int = {word: ii for ii, word in section_idx}
section_to_int

{'강원': 0,
 '경기남부': 1,
 '경기동부': 2,
 '경기북부': 3,
 '경남': 4,
 '광주/전남': 5,
 '대구/경북': 6,
 '대전/충남': 7,
 '부산': 8,
 '서울': 9,
 '세종': 10,
 '울산': 11,
 '인천': 12,
 '전북': 13,
 '제주': 14,
 '충북': 15}

In [5]:
def set_section_idx(section):
    return section_to_int[section]

In [6]:
df['sec_idx']=df['sec_sub'].apply(set_section_idx)

In [7]:
df.sample(10)

,article,title,sec,sec_sub,len,sec_idx
120967,세종시가 ㈜두루셀텍·㈜싸이텍·㈜광스틸·재진가로등㈜ 등 유망 중소기업 4개사와 투자협...,세종시 유망중소기업과 투자협약 체결,전국,세종,158,10
1173069,경남도는 시민들이 밤중에 안심하고 귀가할 수 있는 환경을 조성하기 위해 창원시와 밀...,"경남도, 창원·밀양에 '안심 골목길 조성사업' 추진",전국,경남,537,4
591581,강원지방기상청은 5일 오후 4시를 기해 강릉·동해·삼척·속초·고성·양양 평지에 발효...,"강원영동 건조경보, 평창·인제 주의보 발령",전국,강원,321,0
942023,경기 양평군은 지방도 342호선 용문면 화전리 ~ 마룡리간 지방도로가 완공돼 개통했...,양평군 화전-마룡 간 지방도로 준공,전국,경기동부,601,2
2292438,한국무역협회 대구경북본부는 제53회 무역의 날을 맞아 2016년 유공자 포상 및 수...,"무협, 2016년 유공자 포상·수출의 탑 신청 접수",전국,대구/경북,718,6
426588,대구 서부경찰서는 6일 교통사고 견인요금을 과다 청구한 견인업체 대표 정모(57)씨...,"견인업체, 요금 과다청구했다 무더기 적발",전국,대구/경북,282,6
1286055,칠곡경북대학교병원(원장 박재용)은 31일 대구은행 본점 3층(비지니스룸)에서 대구은...,"칠곡경북대병원-대구銀, 지역민 사회공헌 MOU",전국,대구/경북,481,6
2167320,강원 동해시(심규언)는 15일부터 동해문화예술회관 소공연장과 동해소방서 대회의실 등...,"동해시, 공무원 심폐소생술 교육 실시",전국,강원,673,0
1359268,부산 해운대구청에서 여권·국제운전면허증 등 동시 발급이 가능해 졌다. 해운대구는...,"해운대구, '국제운전면허증' 발급 서비스 시행",전국,부산,396,8
2314357,경북 포항지방해양수산청(청장 윤석홍)은 항내를 출입하는 어선들의 안전항해를 위해 구...,"포항해양청, 방파제 등대 2기 설치",전국,대구/경북,361,6


In [8]:
X = df.article
y = df.sec_idx
print(X.shape)
print(y.shape)

(573195,)
(573195,)


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(458556,)
(114639,)
(458556,)
(114639,)


In [12]:
%%time
from konlpy.tag import Mecab; m = Mecab()
#pos = lambda d: ['/'.join(p) for p in m.pos(d)]
def noun_tagger(text):
    #return [pos[0] for pos in m.pos(text) if (pos[1] in ['SL','SH','SN']) or pos[1].startswith('NN')]
    return [pos[0] for pos in m.pos(text) if pos[1].startswith('NN')]

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 808 µs


### Summary:
    * vect.fit(train) learns the vocabulary of the training data
    * vect.transform(train) uses the fitted vocabulary to build a document term matrix from the training data
    * vect.transform(test) uses the fitted vocabulary to build a document term matrix from the testing data(and ignores tokens it hasn't seen before)

In [13]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(tokenizer=noun_tagger, ngram_range=(1,3), min_df=2, max_df=.5)
X_train_dtm = vect.fit_transform(X_train)

CPU times: user 18min 43s, sys: 16.4 s, total: 19min
Wall time: 19min


In [14]:
X_test_dtm = vect.transform(X_test)

In [15]:
X_train_dtm.shape

(458556, 7912225)

In [16]:
X_test_dtm.shape

(114639, 7912225)

## save vector & dtm

https://stackoverflow.com/questions/29788047/keep-tfidf-result-for-predicting-new-content-using-scikit-for-python
https://stackoverflow.com/questions/32764991/how-do-i-store-a-tfidfvectorizer-for-future-use-in-scikit-learn

In [17]:
import pickle

with open("local_tfidf_vect.pkl", 'wb') as handle:
    pickle.dump(vect, handle)

with open("local_dtm.pkl", 'wb') as handle:
    pickle.dump(X_test_dtm, handle)

In [18]:
# import and instantiate a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB(alpha=.01)

In [19]:
# train the model using X_train_dtm (timing it with an IPython "magic command")
%time nb.fit(X_train_dtm, y_train)

CPU times: user 10.7 s, sys: 1.15 s, total: 11.9 s
Wall time: 11.9 s


MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [20]:
# make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

In [21]:
# calculate accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.89231413393347814

In [22]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_class, target_names=['강원', '경기남부', '경기동부', '경기북부', '경남', '광주/전남', '대구/경북', '대전/충남', '부산', '서울', '세종', '울산', '인천', '전북', '제주', '충북']))

             precision    recall  f1-score   support

         강원       0.89      0.93      0.91     14154
       경기남부       0.77      0.89      0.83      7883
       경기동부       0.87      0.73      0.79      1712
       경기북부       0.92      0.65      0.76      2021
         경남       0.97      0.86      0.91      7641
      광주/전남       0.90      0.89      0.89      8663
      대구/경북       0.89      0.92      0.91     13887
      대전/충남       0.86      0.89      0.88     11217
         부산       0.83      0.92      0.87      7107
         서울       0.96      0.91      0.93      5382
         세종       0.96      0.81      0.88      1676
         울산       0.95      0.89      0.92      5970
         인천       0.92      0.68      0.78      2324
         전북       0.89      0.90      0.89      8447
         제주       0.91      0.92      0.91      5815
         충북       0.95      0.92      0.93     10740

avg / total       0.90      0.89      0.89    114639



In [23]:
import numpy as np
counts = np.bincount(y_pred_class[(y_pred_class!=0) & (y_test==0)])
print(np.argmax(counts))

7


In [24]:
X_test[(y_pred_class==7) & (y_test==0)]

248107     강원대학교(총장 신승호)가 삼성재단에서 주최하는 '삼성드림클래스 2013 여름방학 ...
2354700    북부지방산림청(청장 박기남)은 20일 경기도 양평군 용문양묘사업소에서 낙엽송 우량 ...
1729668    국민건강보험공단은 18일 공단 회의실에서 국립재활원, 도로교통공단, (사)한국장애인...
560938     한국폴리텍대학 원주캠퍼스(학장 임관규)는 지난 22일부터 듀얼공동훈련센터 일학습병행...
151107     한림대학교 최수영 교수(바이오메디컬학과 )가 최근 생화학분자생물학회 신임 회장으로 ...
911238     강원 원주시가 7일 시청 다목적홀에서 정책관리 아카데미를 개최한다.    이번 정책...
1486576    강릉아산병원(병원장 김인구)은 고압산소치료센터를 5월에 오픈하겠다고 20일 밝혔다....
236049     강원 횡성인재육성관(위원장 조원용)은 방학을 맞아 학생들의 학력향상을 위한 학력제고...
1007058    국회 농림축산식품해양수산위원회 소속 새누리당 이이재(동해·삼척)의원은 임업분야 발전...
584368     한림대학교(총장 노건일)는 오는 10일 다양한 연구실험환경을 제공하게 될 한림종합연...
584683     강릉원주대학교 컴퓨터공학과가 교육부의 지역특성화 우수학과 지원사업에 선정됐다.   ...
705853     ▲입학관리본부장 진덕희(생명과학대학 해양분자생명공학과 교수) ▲취업지원본부장 김성준...
191002     ▲성시일 학생처장   [뉴시스 이시간 핫 뉴스] ㆍ 김용판, 청문회 증인 선서 거부...
1915215    존경하는 강원도민 여러분. 사랑하는 강원도 학생, 학부모, 교직원 여러분. 강원도교...
580093     강원도교육청(교육감 민병희)은 교육부에서 주최하는 '제4회 Wee 희망 대상' 3개...
1439054    백환조 강원대학교(총장 신승호) 창업지원단장이 최근 대전에서 열린 '제2회 정기회'...
608883     강원 태백시는 개학기를 

In [25]:
df[df.index==904206]

,article,title,sec,sec_sub,len,sec_idx
904206,강원 강릉원주대학교(총장 전방욱)는 중어중문학과 졸업생 이아름(25·여)·서혜리(2...,강릉원주대 이아름·서혜리씨 중국정부 초청장학생 합격,전국,강원,445,0


In practice Multinomial NB can be better than Linear SVM in some situations and svm better than NB on other datasets. Apparently t's possible to combine the two approaches to get a very good baseline: projects:nbsvm - Sida I. Wang

You can also try non-linear SVMs but often the quadratic complexity of the SMO algorithm (for instance as implemented in libsvm) makes it not practical on datasets with more than 5000 documents. Instead on prefer to use liblinear than can only train linear SVM on large datasets.

https://www.quora.com/What-are-the-differences-similarities-between-SVM-Naive-Bayes-for-binary-text-classification-wrt-how-they-are-processing-the-features

http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC

In [26]:
from sklearn import svm
#clf = svm.SVC(decision_function_shape='ovo')
clfrSVM=svm.LinearSVC(C=1)

In [27]:
%time clfrSVM.fit(X_train_dtm, y_train)

CPU times: user 4min 46s, sys: 1.01 s, total: 4min 47s
Wall time: 4min 47s


LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [28]:
y_pred_class_svm = clfrSVM.predict(X_test_dtm)

In [29]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class_svm)

0.97676183497762537

In [30]:
print(classification_report(y_test, y_pred_class_svm, target_names=['강원', '경기남부', '경기동부', '경기북부', '경남', '광주/전남', '대구/경북', '대전/충남', '부산', '서울', '세종', '울산', '인천', '전북', '제주', '충북']))

             precision    recall  f1-score   support

         강원       0.99      0.99      0.99     14154
       경기남부       0.95      0.96      0.96      7883
       경기동부       0.98      0.96      0.97      1712
       경기북부       0.95      0.90      0.93      2021
         경남       0.98      0.98      0.98      7641
      광주/전남       0.97      0.98      0.98      8663
      대구/경북       0.98      0.98      0.98     13887
      대전/충남       0.96      0.97      0.97     11217
         부산       0.97      0.97      0.97      7107
         서울       0.98      0.97      0.97      5382
         세종       0.96      0.91      0.93      1676
         울산       0.99      0.99      0.99      5970
         인천       0.97      0.95      0.96      2324
         전북       0.99      0.99      0.99      8447
         제주       0.99      0.99      0.99      5815
         충북       0.99      0.98      0.99     10740

avg / total       0.98      0.98      0.98    114639



https://stackoverflow.com/questions/40115043/no-space-left-on-device-error-while-fitting-sklearn-model

In [31]:
with open("local_svm_clf.pkl", 'wb') as handle:
    pickle.dump(clfrSVM, handle)

In [29]:
with open("kbo_nb_clf.pkl", 'wb') as handle:
    pickle.dump(nb, handle)